In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 4. Camada Gold - Agregações de Negócio (KPIs)
# MAGIC 
# MAGIC **Objetivo:** Criar tabelas agregadas com métricas de negócio a partir da camada Silver.

# COMMAND ----------

from pyspark.sql.functions import col, sum, count, avg, desc, explode, from_json
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# Define o catálogo a ser usado
spark.sql("USE CATALOG movies_catalog")

# COMMAND ----------

# Carrega a tabela limpa da camada Silver
df_silver = spark.read.table("silver.movies_cleaned")

# --- Agregação 1: Lucro por Filme ---
print("Calculando o lucro por filme...")
df_lucro_por_filme = df_silver \
    .withColumn("lucro", col("receita") - col("orcamento")) \
    .select("titulo_original", "ano_lancamento", "orcamento", "receita", "lucro") \
    .orderBy(desc("lucro"))

df_lucro_por_filme.write.mode("overwrite").saveAsTable("gold.agg_lucro_por_filme")
print("Tabela 'gold.agg_lucro_por_filme' criada com sucesso.")
display(df_lucro_por_filme)


# --- Agregação 2: Quantidade de Filmes por Ano ---
print("\nCalculando a quantidade de filmes por ano...")
df_filmes_por_ano = df_silver.groupBy("ano_lancamento") \
                             .agg(count("id").alias("quantidade_filmes")) \
                             .orderBy(desc("ano_lancamento"))

df_filmes_por_ano.write.mode("overwrite").saveAsTable("gold.agg_filmes_por_ano")
print("Tabela 'gold.agg_filmes_por_ano' criada com sucesso.")
display(df_filmes_por_ano)


# --- Agregação 3: Receita Média por Gênero ---
# A coluna 'genres' parece ser uma string de um array de JSONs. Precisamos tratá-la.
print("\nCalculando a receita média por gênero...")

# Schema para extrair o nome do gênero do JSON
genre_schema = ArrayType(StructType([StructField("name", StringType())]))

df_generos = df_silver.withColumn("genres_array", from_json(col("genres"), genre_schema)) \
                      .withColumn("genre_struct", explode(col("genres_array"))) \
                      .withColumn("genero", col("genre_struct.name")) \
                      .select("genero", "receita")

df_receita_por_genero = df_generos.groupBy("genero") \
                                  .agg(
                                      avg("receita").alias("receita_media"),
                                      count("*").alias("quantidade_filmes")
                                  ) \
                                  .orderBy(desc("receita_media"))

df_receita_por_genero.write.mode("overwrite").saveAsTable("gold.agg_receita_media_por_genero")
print("Tabela 'gold.agg_receita_media_por_genero' criada com sucesso.")
display(df_receita_por_genero)